In [98]:
%config IPCompleter.greedy=True

In [99]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.linear_model import LogisticRegression

%matplotlib inline

# Exercise 12

In [100]:
credit_df = pd.read_csv('./data/Credit.csv')
credit_df.drop('Unnamed: 0', axis=1, inplace=True)

In [101]:
col_names = ['Married', 'Ethnicity', 'Gender', 'Student']

for col in col_names:
    try:
        dummies = pd.get_dummies(credit_df[col], prefix=col)
        credit_df.drop(col, axis=1, inplace=True)
        credit_df = pd.concat([credit_df, dummies], axis=1)
    except KeyError as error:
        print('Column {name} does not exist!'.format(name=col))

In [102]:
credit_df

,Income,Limit,Rating,Cards,Age,Education,Balance,Married_No,Married_Yes,Ethnicity_African American,Ethnicity_Asian,Ethnicity_Caucasian,Gender_Female,Gender_Male,Student_No,Student_Yes
0,14.891,3606,283,2,34,11,333,0,1,0,0,1,0,1,1,0
1,106.025,6645,483,3,82,15,903,0,1,0,1,0,1,0,0,1
2,104.593,7075,514,4,71,11,580,1,0,0,1,0,0,1,1,0
3,148.924,9504,681,3,36,11,964,1,0,0,1,0,1,0,1,0
4,55.882,4897,357,2,68,16,331,0,1,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,12.096,4100,307,3,32,13,560,0,1,0,0,1,0,1,1,0
396,13.364,3838,296,5,65,17,480,1,0,1,0,0,0,1,1,0
397,57.872,4171,321,5,67,12,138,0,1,0,0,1,1,0,1,0
398,37.728,2525,192,1,44,13,0,0,1,0,0,1,0,1,1,0


## a) income > 50

In [103]:
income_df = credit_df.copy(deep=True)

In [104]:
income_df['Income > 50'] = income_df.apply(lambda row: 1 if row['Income'] > 50 else 0, axis=1)
income_df

,Income,Limit,Rating,Cards,Age,Education,Balance,Married_No,Married_Yes,Ethnicity_African American,Ethnicity_Asian,Ethnicity_Caucasian,Gender_Female,Gender_Male,Student_No,Student_Yes,Income > 50
0,14.891,3606,283,2,34,11,333,0,1,0,0,1,0,1,1,0,0
1,106.025,6645,483,3,82,15,903,0,1,0,1,0,1,0,0,1,1
2,104.593,7075,514,4,71,11,580,1,0,0,1,0,0,1,1,0,1
3,148.924,9504,681,3,36,11,964,1,0,0,1,0,1,0,1,0,1
4,55.882,4897,357,2,68,16,331,0,1,0,0,1,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,12.096,4100,307,3,32,13,560,0,1,0,0,1,0,1,1,0,0
396,13.364,3838,296,5,65,17,480,1,0,1,0,0,0,1,1,0,0
397,57.872,4171,321,5,67,12,138,0,1,0,0,1,1,0,1,0,1
398,37.728,2525,192,1,44,13,0,0,1,0,0,1,0,1,1,0,0


In [105]:
income_features = list(income_df.columns)[:-1]
income_result = ['Income > 50']

In [106]:
X_income, y_income = income_df[income_features], income_df[income_result]

In [107]:
def split_dataset(df, feat, res):
    X = df[feat]
    y = df[res]
    return train_test_split(X, y, test_size=0.2, random_state=13)

In [108]:
X_income_train, X_income_test, y_income_train, y_income_test = split_dataset(income_df, income_features, income_result)

In [109]:
income_model = LogisticRegression(C=1e9, fit_intercept=False, solver='liblinear', penalty='l2', max_iter=10000)
income_model.fit(X_income_train, y_income_train.to_numpy().ravel())
income_model_acc = income_model.score(X_income_test, y_income_test.to_numpy().ravel())
income_model_acc

0.9875

## b) How many credit cards a person has?

In [110]:
cc_df = credit_df.copy(deep=True)

In [111]:
cc_features = list(cc_df)
cc_features.remove('Cards')
cc_result = ['Cards']

In [112]:
X_cc, y_cc = cc_df[cc_features], cc_df[cc_result]
X_cc_train, X_cc_test, y_cc_train, y_cc_test = split_dataset(cc_df, cc_features, cc_result)

In [116]:
cc_model = LogisticRegression(C=1e9, fit_intercept=False, solver='liblinear', max_iter=10000, multi_class='auto', penalty='l1')
cc_model.fit(X_cc_train, y_cc_train.to_numpy().ravel())
cc_model_acc = cc_model.score(X_cc_test, y_cc_test.to_numpy().ravel())
cc_model_acc

0.3625

# Exercise 13

## a) Income > 50

### KNN